In [1]:
import re
import string
import itertools
from collections import Counter

import pandas as pd
import numpy as np

# import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import spacy

from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from tqdm.auto import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/Module_5_Lecture_1_Class_amazon_product_reviews.csv', index_col='Id')

In [3]:
# Dataset preview

df.head(3)

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
Id,,,,,,,,,
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [4]:
# Basic dataset info 

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568454 entries, 1 to 568454
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ProductId               568454 non-null  object
 1   UserId                  568454 non-null  object
 2   ProfileName             568428 non-null  object
 3   HelpfulnessNumerator    568454 non-null  int64 
 4   HelpfulnessDenominator  568454 non-null  int64 
 5   Score                   568454 non-null  int64 
 6   Time                    568454 non-null  int64 
 7   Summary                 568427 non-null  object
 8   Text                    568454 non-null  object
dtypes: int64(4), object(5)
memory usage: 43.4+ MB


In [5]:
# For this lesson, let's ignore neutral reviews
# We'll consider reviews with a score 3 as neutral

df = df.loc[df['Score']!=3]

df.shape

(525814, 9)

In [6]:
df['sentiment'] = [1 if score in [4, 5] else 0 for score in df['Score']]

In [7]:
# Number of identical records

df.duplicated().sum()

255

In [8]:
# Droping duplicated records

df = df.drop_duplicates().reset_index(drop=True)

# Number of unique records

df.shape

(525559, 10)

In [9]:
# Checking for the identical reviews of different versions of the same product

df.groupby(['UserId', 'Time', 'Text']).count().sort_values('ProductId', ascending=False).head(10)

ProductId  \
UserId         Time       Text                                                            
A3TVZM3ZIXG8YW 1291420800 This review will make me sound really stupid, b...        182   
A36JDIN9RAAIEC 1292976000 I have two cats, one 6 and one 2 years old. Bot...         51   
A1TMAVN4CEM8U8 1336348800 Diamond Almonds<br />Almonds are a good source ...         43   
A1UQBFCERIP7VJ 1321401600 Stash Chamomile Herbal Tea is tea bags with dri...         38   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...         35   
A25C5MVVCIYT5D 1304726400 I understand all the complaints about Science D...         34   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...         34   
A24PZR4W555WQI 1294617600 My dogs and I love this food. They never leave ...         28   
A3PJZ8TU8FDQ1K 1231718400 I'm addicted to salty and tangy flavors, so whe...         28   
A3A1OA237FOZFK 1296950400 I was a little hesitant to try these, especiall...         28   

                                                                              ProfileName  \
UserId         Time       Text                                                              
A3TVZM3ZIXG8YW 1291420800 This review will make me sound really stupid, b...          182   
A36JDIN9RAAIEC 1292976000 I have two cats, one 6 and one 2 years old. Bot...           51   
A1TMAVN4CEM8U8 1336348800 Diamond Almonds<br />Almonds are a good source ...           43   
A1UQBFCERIP7VJ 1321401600 Stash Chamomile Herbal Tea is tea bags with dri...           38   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...           35   
A25C5MVVCIYT5D 1304726400 I understand all the complaints about Science D...           34   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...           34   
A24PZR4W555WQI 1294617600 My dogs and I love this food. They never leave ...           28   
A3PJZ8TU8FDQ1K 1231718400 I'm addicted to salty and tangy flavors, so whe...           28   
A3A1OA237FOZFK 1296950400 I was a little hesitant to try these, especiall...           28   

                                                                              HelpfulnessNumerator  \
UserId         Time       Text                                                                       
A3TVZM3ZIXG8YW 1291420800 This review will make me sound really stupid, b...                   182   
A36JDIN9RAAIEC 1292976000 I have two cats, one 6 and one 2 years old. Bot...                    51   
A1TMAVN4CEM8U8 1336348800 Diamond Almonds<br />Almonds are a good source ...                    43   
A1UQBFCERIP7VJ 1321401600 Stash Chamomile Herbal Tea is tea bags with dri...                    38   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...                    35   
A25C5MVVCIYT5D 1304726400 I understand all the complaints about Science D...                    34   
A29JUMRL1US6YP 1278201600 The pet food industry can be one of the most in...                    34   
A24PZR4W555WQI 1294617600 My dogs and I love this food. They never leave ...                    28   
A3PJZ8TU8FDQ1K 1231718400 I'm addicted to salty and tangy flavors, so whe...                    28   
A3A1OA237FOZFK 1296950400 I was a little hesitant to try these, especiall...                    28   

                                                                              HelpfulnessDenominator  \
UserId         Time       Text                                                                         
A3TVZM3ZIXG8YW 1291420800 This review will make me sound really stupid, b...                     182   
A36JDIN9RAAIEC 1292976000 I have two cats, one 6 and one 2 years old. Bot...                      51   
A1TMAVN4CEM8U8 1336348800 Diamond Almonds<br />Almonds are a good source ...                      43   
A1UQBFCERIP7VJ 1321401600 Stash Chamomile Herbal Tea is tea bags with dri...                      38   
A29JUMRL1US6YP 1278

In [10]:
search_text = "I have two cats, one 6 and one 2 years old. Both are indoor cats in excellent health. I saw the negative review and talked to my vet about it. I've also asked a number of veterinary professionals what to feed my cats and they all answer the same thing: Science Diet. Sure, you'll see stories of how one person's cat had issues, but even if that's 100% true, it's 1 case out of millions. Science and fact aren't based on someone's experience.<br /><br />So my point is, I love my cats and I'm very concerned about their health. I trust people who actually have medical degrees and experience with a wide range of animals. My only caution is do not fall for some hype or scare tactic that recommends some unproven or untested food or some fad diet for your pet. Don't listen to me, don't listen to the negative review. ASK YOUR VET what they recommend, and follow their instructions. My guess is you'll end up buying the Science Diet anyhow."
duplicates_example = df.loc[
    (df['UserId']=='A36JDIN9RAAIEC') &
    (df['Time']==1292976000) &
    (df['Text']==search_text)
]

duplicates_example

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment
48291,B003ANFMY8,A36JDIN9RAAIEC,Jon,3,3,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
48301,B003ANFMY8,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
48302,B003ANFMY8,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,Don't fall prey to fads and anecdotal reviews,"I have two cats, one 6 and one 2 years old. Bo...",1
88934,B003WK0D8O,A36JDIN9RAAIEC,Jon,3,3,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
88944,B003WK0D8O,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
88945,B003WK0D8O,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,Don't fall prey to fads and anecdotal reviews,"I have two cats, one 6 and one 2 years old. Bo...",1
89993,B0002MLA5K,A36JDIN9RAAIEC,Jon,3,3,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
90003,B0002MLA5K,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1
90004,B0002MLA5K,A36JDIN9RAAIEC,Jon,2,2,5,1292976000,Don't fall prey to fads and anecdotal reviews,"I have two cats, one 6 and one 2 years old. Bo...",1
133472,B003MA8P02,A36JDIN9RAAIEC,Jon,3,3,5,1292976000,"Great product, but trust your vet not the hype","I have two cats, one 6 and one 2 years old. Bo...",1


In [11]:
# Droping the same reviews

df = df.drop_duplicates(subset={"UserId", "Time","Text"})

# Final size 

df.shape

(364133, 10)

In [12]:
# Contractions. Source http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [13]:
# get stop-words from the nltk library
# using set to make words search faster

stop_words = set(stopwords.words('english')).union({'also', 'would', 'much', 'many'})

In [14]:
negations = {
    'aren',
    "aren't",
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'isn',
    "isn't",
    'mightn',
    "mightn't",
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'no',
    'nor',
    'not',
    'shan',
    "shan't",
    'shouldn',
    "shouldn't",
    'wasn',
    "wasn't",
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't"
}

In [15]:
# removing negations from the stop-words list

stop_words = stop_words.difference(negations)

In [16]:
stemmer = PorterStemmer()

In [17]:
nlp = spacy.load("en_core_web_sm", disable = ['parser','ner'])

In [18]:
# function to clean text
def normalize_text(raw_review):
    
    # Remove html tags
    text = re.sub("<[^>]*>", " ", raw_review) # match <> and everything in between. [^>] - match everything except >
    
    # Remove emails
    text = re.sub("\S*@\S*[\s]+", " ", text) # match non-whitespace characters, @ and a whitespaces in the end
    
    # remove links
    text = re.sub("https?:\/\/.*?[\s]+", " ", text) # match http, s - zero or once, //, 
                                                    # any char 0-unlimited, whitespaces in the end
        
     # Convert to lower case, split into individual words
    text = text.lower().split()
    
    # Replace contractions with their full versions
    text = [contractions.get(word) if word in contractions else word 
            for word in text]
   
    # Re-splitting for the correct stop-words extraction
    text = " ".join(text).split()    
    
    # Remove stop words
    text = [word for word in text if not word in stop_words]

    text = " ".join(text)
    
    # Remove non-letters        
    text = re.sub("[^a-zA-Z' ]", "", text) # match everything except letters and '


    # Stem words. Need to define porter stemmer above
    # text = [stemmer.stem(word) for word in text.split()]

    # Lemmatize words. Need to define lemmatizer above
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if len(token.lemma_) > 1 ])
    
    # Remove excesive whitespaces
    text = re.sub("[\s]+", " ", text)    
    
    # Join the words back into one string separated by space, and return the result.
    return text

In [19]:
text = 'On a quest for the perfedc1112t,,, !!!! <br />%%2%% popcorn to compliment the Whirley Pop.  Don\'t get older, I\'m beginning to appreciate the more "natural" popcorn varieties, and I suppose that\'s what attracted me to the Arrowhead Mills Organic Yellow Popcorn.<br /> <br />I\'m no "organic" food expert.  I just wanted some good tasting popcorn.  And, I feel like that\'s what I got.  Using the Whirley Pop, with a very small amount of oil, I\'ve had great results.'

print('Original text', text, '#'*30, sep='\n\n')
print('\nNormalized text', normalize_text(text), sep='\n\n')

Original text

On a quest for the perfedc1112t,,, !!!! <br />%%2%% popcorn to compliment the Whirley Pop.  Don't get older, I'm beginning to appreciate the more "natural" popcorn varieties, and I suppose that's what attracted me to the Arrowhead Mills Organic Yellow Popcorn.<br /> <br />I'm no "organic" food expert.  I just wanted some good tasting popcorn.  And, I feel like that's what I got.  Using the Whirley Pop, with a very small amount of oil, I've had great results.

##############################

Normalized text

quest perfedct popcorn compliment whirley pop not get old begin appreciate natural popcorn variety suppose attract arrowhead mill organic yellow popcorn no organic food expert want good tasting popcorn and feel like get use whirley pop small amount oil great result


In [20]:
# Slicing dataset for demonstrative purposes

df = df.groupby('sentiment').sample(2500, random_state=42)

# Note: without resetting an index we slice over the original Id`s
df.shape

(5000, 10)

In [21]:
df['text_normalized'] = df['Text'].progress_apply(normalize_text)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [22]:
train_idxs = df.sample(frac=0.8, random_state=42).index
test_idxs = [idx for idx in df.index if idx not in train_idxs]

In [23]:
X_train = df.loc[train_idxs, 'text_normalized']
X_test = df.loc[test_idxs, 'text_normalized']

y_train = df.loc[train_idxs, 'sentiment']
y_test = df.loc[test_idxs, 'sentiment']

In [24]:
# Creating and training a CountVectorizer object 

vect = CountVectorizer().fit(X_train)

len(vect.vocabulary_)

12650

In [25]:
# features examples

vect.get_feature_names_out()[:5]

array(['aa', 'ab', 'aback', 'abandon', 'abba'], dtype=object)

In [26]:
# transform the documents in the training data to a document-term matrix

X_train_vectorized = vect.transform(X_train)
X_train_vectorized.shape

(4000, 12650)

In [27]:
# Resulted features representation is a sparse matrix

X_train_vectorized

<4000x12650 sparse matrix of type '<class 'numpy.int64'>'
	with 138253 stored elements in Compressed Sparse Row format>

In [28]:
model = LogisticRegression(random_state=42)
model.fit(X_train_vectorized, y_train)

LogisticRegression(random_state=42)

In [29]:
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8371428571428571


In [30]:
def get_preds(text_column, algorithm, ngrams=(1,1)):
    
    X_train = df.loc[train_idxs, text_column]
    X_test = df.loc[test_idxs, text_column]

    y_train = df.loc[train_idxs, 'sentiment']
    y_test = df.loc[test_idxs, 'sentiment']
    
    if algorithm == 'cv':
        vect = CountVectorizer(ngram_range=ngrams).fit(X_train)
    elif algorithm == 'tfidf':
        vect = TfidfVectorizer(ngram_range=ngrams).fit(X_train)
    else:
        raise ValueError('Select correct algorithm: `cv` or `tfidf`')
            
    print('Vocabulary length: ', len(vect.vocabulary_))
    
    # transform the documents in the training data to a document-term matrix

    X_train_vectorized = vect.transform(X_train)
    print('Document-term matrix shape:', X_train_vectorized.shape)
    
    model = LogisticRegression(random_state=42)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))

    print('AUC: ', roc_auc_score(y_test, predictions))

In [31]:
get_preds('Text', 'cv')

Vocabulary length:  14214
Document-term matrix shape: (4000, 14214)
AUC:  0.8522305764411028


In [32]:
get_preds('text_normalized', 'tfidf')

Vocabulary length:  12650
Document-term matrix shape: (4000, 12650)
AUC:  0.8466165413533835


In [33]:
get_preds('Text', 'tfidf')

Vocabulary length:  14214
Document-term matrix shape: (4000, 14214)
AUC:  0.8407017543859648


In [34]:
get_preds('text_normalized', 'cv', (1,2))

Vocabulary length:  127773
Document-term matrix shape: (4000, 127773)
AUC:  0.8695739348370928


In [35]:
get_preds('text_normalized', 'tfidf', (1,2))

Vocabulary length:  127773
Document-term matrix shape: (4000, 127773)
AUC:  0.8461654135338346


In [36]:
get_preds('text_normalized', 'cv', (2,2))

Vocabulary length:  115123
Document-term matrix shape: (4000, 115123)
AUC:  0.7773934837092732


In [37]:
get_preds('Text', 'cv', (2,2))

Vocabulary length:  132784
Document-term matrix shape: (4000, 132784)
AUC:  0.8374436090225565


In [38]:
get_preds('Text', 'tfidf', (2,2))

Vocabulary length:  132784
Document-term matrix shape: (4000, 132784)
AUC:  0.8363408521303258
